# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
import time
from itertools import islice
from dataclasses import dataclass
import torchvision
from torchvision.models import densenet161, DenseNet161_Weights, vit_b_16, ViT_B_16_Weights, densenet121, DenseNet121_Weights
import os
import sys
from pathlib import Path
from torchinfo import summary

In [4]:
from CheXpert.race_prediction.dataset import CheXpertRaceDataset
from CheXpert.disease_prediction.dataset import CheXpertDiseaseDataset
from CheXpert.shared_utils import vprint, to_gpu
from CheXpert import shared_utils
from CheXpert.disease_prediction.utils import Configs as disease_configs
from CheXpert.race_prediction.utils import Configs as race_configs

# Configs

In [15]:
@dataclass
class Configs:
    DATA_DIR = os.path.join("data", "CheXpert", "CheXpert-v1.0-small")
    DISEASE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "disease_prediction", "trained_models")
    RACE_TRAINED_MODELS_DIR = os.path.join("CheXpert", "race_prediction", "trained_models")
    VALID_LABELS_FILENAME = "valid.csv"
    DEMO_FILENAME = "CHEXPERT DEMO.csv"
    DISEASE_CLASSES = disease_configs.ANNOTATIONS_COLUMNS
    RACE_CLASSES = race_configs.ANNOTATIONS_COLUMNS
    NUM_DISEASE_CLASSES = disease_configs.NUM_CLASSES
    NUM_RACE_CLASSES = race_configs.NUM_CLASSES
    BATCH_SIZE = 4
    SEED = 123
    VERBOSE = 1
    VALID_SIZE_DEBUG = 10

In [16]:
shared_utils.set_seed(Configs.SEED)

In [17]:
if torch.cuda.is_available():
    vprint(f"Memory info: {torch.cuda.mem_get_info()[0]/10e8:.1f} GB free GPU.", Configs)
else: 
    vprint(f"No GPU Memory.", Configs)

2022-07-21 17:16: No GPU Memory.


In [18]:
valid_transform = transforms.Compose([
    transforms.Resize((320,320)),
    transforms.ToTensor(), 
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Disease Prediction 

In [19]:
disease_valid_dataset = CheXpertDiseaseDataset(data_dir=Configs.DATA_DIR, 
                                               labels_filename=Configs.VALID_LABELS_FILENAME,
                                               transform=valid_transform)
disease_valid_dataset.df_labels = disease_valid_dataset.df_labels[:Configs.VALID_SIZE_DEBUG] # hack for speed debugging
disease_valid_dataloader = DataLoader(disease_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
len(disease_valid_dataset)

10

In [20]:
_, _, files = next(os.walk(Configs.DISEASE_TRAINED_MODELS_DIR))
disease_trained_models = [os.path.join(Configs.DISEASE_TRAINED_MODELS_DIR, file) for file in files]
len(disease_trained_models)

1

In [21]:
model = densenet121()
num_features = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Linear(num_features, num_features, bias=True),
    nn.ReLU(),
    nn.Dropout(p=0.1),
    nn.Linear(in_features=num_features, out_features=Configs.NUM_DISEASE_CLASSES, bias=True)
)
model.eval()
not model.training

True

In [22]:
model, results, _, _ = shared_utils.load_statedict(model, disease_trained_models[0], Configs)

2022-07-21 17:16: Loading model - CheXpert\disease_prediction\trained_models\2022_07_12-18_47__densenet121_aug__epoch-5__iter-12659__batch_size-16__trainLastLoss-0.3754__validAUC-0.8899.dict


In [23]:
labels, outputs = shared_utils.get_metric_tensors(model, disease_valid_dataloader, Configs,
                                                  apply_on_outputs=lambda x: torch.sigmoid(x),
                                                  by_study=False, challenge_ann_only=None)

In [24]:
df_res_disease = pd.DataFrame(columns=Configs.DISEASE_CLASSES, 
                              data=[shared_utils.auc_score(labels, outputs, per_class=True),],
                             index=['AUC'])
df_res_disease

Only one class present in y_true. ROC AUC score is not defined in that case.


,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
AUC,1.0,0.875,-1,1.0,1.0


# Race Prediction

In [27]:
# Create data loaders.
race_valid_dataset = CheXpertRaceDataset(data_dir=Configs.DATA_DIR, demo_filename=Configs.DEMO_FILENAME,
                                       labels_filename=Configs.VALID_LABELS_FILENAME,
                                       transform=valid_transform)
race_valid_dataloader = DataLoader(race_valid_dataset, batch_size=Configs.BATCH_SIZE, shuffle=False)
Configs.TRAIN_LOADER_SIZE = len(race_valid_dataloader)
len(race_valid_dataset)

172

In [36]:
_, _, files = next(os.walk(Configs.RACE_TRAINED_MODELS_DIR))
race_trained_models = [os.path.join(Configs.RACE_TRAINED_MODELS_DIR, file) for file in files]
race_model_versions = [p.split('__')[1] for p in race_trained_models]
len(race_trained_models)

7

In [47]:
df_res_race = pd.DataFrame(columns=Configs.RACE_CLASSES)
df_res_race

,White,Asian,Black,Hispanic


In [48]:
for model_version, model_path in zip(model_versions, race_trained_models):
    model = densenet121()
    
    if "shallow" in model_version:
        shallow_denseblock = int(model_version.split('_')[2][10:])
        layer_offset = 3 + 2 * shallow_denseblock
        num_features = model.features[layer_offset].norm.num_features
        model = model.features[:layer_offset]
        classifier_module = nn.Sequential(
            nn.BatchNorm2d(num_features, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1, 1)),   
            nn.Flatten(start_dim=1),
            nn.Linear(in_features=num_features, out_features=num_features, bias=True),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
        model.add_module('classifier', classifier_module)
    else:
        num_features = model.classifier.in_features
        model.classifier = nn.Sequential(
            nn.Linear(num_features, num_features, bias=True),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(in_features=num_features, out_features=Configs.NUM_RACE_CLASSES, bias=True))
    model.eval()
    model, results, _, _ = shared_utils.load_statedict(model, model_path, Configs)
    labels, outputs = shared_utils.get_metric_tensors(model, race_valid_dataloader, Configs,
                                                      apply_on_outputs=lambda x: torch.softmax(x, dim=1),
                                                      by_study=False, challenge_ann_only=None)
    auc_scores = shared_utils.auc_score(labels, outputs, per_class=True)
    df_res_race.loc[model_version] = auc_scores

df_res_disease   

2022-07-21 19:31: Loading model - CheXpert\race_prediction\trained_models\2022_07_17-14_07__densenet121_race__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2842__validAUC-0.9196.dict
2022-07-21 19:31: Loading model - CheXpert\race_prediction\trained_models\2022_07_18-10_11__densenet121_race_classifier_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.732__validAUC-0.6978.dict
2022-07-21 19:34: Loading model - CheXpert\race_prediction\trained_models\2022_07_18-17_32__densenet121_race_denseblock4_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.496__validAUC-0.8621.dict
2022-07-21 19:37: Loading model - CheXpert\race_prediction\trained_models\2022_07_18-21_46__densenet121_race_denseblock3_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.3124__validAUC-0.8853.dict
2022-07-21 19:39: Loading model - CheXpert\race_prediction\trained_models\2022_07_19-02_27__densenet121_race_denseblock2_freezed__epoch-9__iter-10266__batch_size-16__trainLastLoss-0.2811_

RuntimeError: Error(s) in loading state_dict for Sequential:
	Missing key(s) in state_dict: "conv0.weight", "norm0.weight", "norm0.bias", "norm0.running_mean", "norm0.running_var", "denseblock1.denselayer1.norm1.weight", "denseblock1.denselayer1.norm1.bias", "denseblock1.denselayer1.norm1.running_mean", "denseblock1.denselayer1.norm1.running_var", "denseblock1.denselayer1.conv1.weight", "denseblock1.denselayer1.norm2.weight", "denseblock1.denselayer1.norm2.bias", "denseblock1.denselayer1.norm2.running_mean", "denseblock1.denselayer1.norm2.running_var", "denseblock1.denselayer1.conv2.weight", "denseblock1.denselayer2.norm1.weight", "denseblock1.denselayer2.norm1.bias", "denseblock1.denselayer2.norm1.running_mean", "denseblock1.denselayer2.norm1.running_var", "denseblock1.denselayer2.conv1.weight", "denseblock1.denselayer2.norm2.weight", "denseblock1.denselayer2.norm2.bias", "denseblock1.denselayer2.norm2.running_mean", "denseblock1.denselayer2.norm2.running_var", "denseblock1.denselayer